This notebook explores a machine learning algorithm to predict the stock prices of SPY, the S&P 500 ETF, and is intended to utilize functions that can be easily translated to a python executable.

In [1]:
# installing dependencies
!pip install numpy
!pip install pandas
!pip install torch
!pip install matplotlib
!pip install alpha_vantage
!pip install scikit-learn
!pip install pandas_market_calendars
!pip install lxml

In [2]:
# import libararies
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from alpha_vantage.timeseries import TimeSeries 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import pandas_market_calendars as mcal
import time

In [3]:
# config file (placing here for now, some fields will change on later impplementations)

config = {
    "alpha_vantage": {
        "key": "2JMCN347HZ3BU9RC", 
        "symbol": "SPY",
        "outputsize": "full",
        "key_adjusted_close": "5. adjusted close",
    },
    "data": {
        "window_size": 30,
        "train_split_size": 1,
    }, 
    "plots": {
        "xticks_interval": 90, # show a date every 90 days
        "color_actual": "#001f3f",
        "color_train": "#3D9970",
        "color_val": "#0074D9",
        "color_pred_train": "#3D9970",
        "color_pred_val": "#0074D9",
        "color_pred_test": "#FF4136",
    },
    "model": {
        "input_size": 1, # since for now we are only using close price
        "num_lstm_layers": 2,
        "lstm_size": 32,
        "dropout": 0.2,
    },
    "training": {
        "device": "cpu",
        "batch_size": 64,
        "num_epoch": 50,
        "epoch_stop": 10,
        "learning_rate": 0.01,
        "scheduler_step_size": 40,
    }
}

In [4]:
# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# tickers = np.array(sp500[0]['Symbol'])
# if ('BF.B' in tickers)
#     tickers[tickers.index('BF.B')] = 'BF-B'

start = time.time()

tickers = ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP',
       'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK',
       'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA',
       'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO',
       'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX',
       'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK',
       'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO',
       'BF-B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH',
       'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE',
       'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG',
       'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX',
       'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP',
       'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST',
       'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA',
       'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISH',
       'DIS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DD',
       'DXC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'ELV',
       'LLY', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EQT', 'EFX', 'EQIX',
       'EQR', 'ESS', 'EL', 'ETSY', 'RE', 'EVRG', 'ES', 'EXC', 'EXPE',
       'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FICO', 'FAST', 'FRT', 'FDX',
       'FITB', 'FRC', 'FSLR', 'FE', 'FIS', 'FISV', 'FLT', 'FMC', 'F',
       'FTNT', 'FTV', 'FOXA', 'FOX', 'BEN', 'FCX', 'GRMN', 'IT', 'GEHC',
       'GEN', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GL', 'GPN',
       'GS', 'HAL', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES',
       'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ',
       'HUM', 'HBAN', 'HII', 'IBM', 'IEX', 'IDXX', 'ITW', 'ILMN', 'INCY',
       'IR', 'PODD', 'INTC', 'ICE', 'IFF', 'IP', 'IPG', 'INTU', 'ISRG',
       'IVZ', 'INVH', 'IQV', 'IRM', 'JBHT', 'JKHY', 'J', 'JNJ', 'JCI',
       'JPM', 'JNPR', 'K', 'KDP', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI',
       'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LDOS',
       'LEN', 'LNC', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LYB', 'MTB',
       'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MTCH',
       'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'META', 'MET', 'MTD', 'MGM',
       'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'MOH', 'TAP', 'MDLZ',
       'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP',
       'NFLX', 'NWL', 'NEM', 'NWSA', 'NWS', 'NEE', 'NKE', 'NI', 'NDSN',
       'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI',
       'ORLY', 'OXY', 'ODFL', 'OMC', 'ON', 'OKE', 'ORCL', 'OGN', 'OTIS',
       'PCAR', 'PKG', 'PARA', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PEP',
       'PKI', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL',
       'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PTC',
       'PSA', 'PHM', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX',
       'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL', 'ROP',
       'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE',
       'NOW', 'SHW', 'SPG', 'SWKS', 'SJM', 'SNA', 'SEDG', 'SO', 'LUV',
       'SWK', 'SBUX', 'STT', 'STLD', 'STE', 'SYK', 'SYF', 'SNPS', 'SYY',
       'TMUS', 'TROW', 'TTWO', 'TPR', 'TRGP', 'TGT', 'TEL', 'TDY', 'TFX',
       'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG',
       'TRV', 'TRMB', 'TFC', 'TYL', 'TSN', 'USB', 'UDR', 'ULTA', 'UNP',
       'UAL', 'UPS', 'URI', 'UNH', 'UHS', 'VLO', 'VTR', 'VRSN', 'VRSK',
       'VZ', 'VRTX', 'VFC', 'VTRS', 'VICI', 'V', 'VMC', 'WAB', 'WBA',
       'WMT', 'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC',
       'WRK', 'WY', 'WHR', 'WMB', 'WTW', 'GWW', 'WYNN', 'XEL', 'XYL',
       'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']

In [5]:
# to account for API limitations (on the weekend)

# index = tickers.index('L')
# tickers = tickers[(index+1):]
len(tickers)

503

In [6]:
today = datetime.now()
next_weeks = mcal.date_range(mcal.get_calendar('NYSE').schedule(start_date=today, end_date=(today+relativedelta(months=1))), frequency='1D')
next_weeks = [date.strftime('%Y-%m-%d') for date in next_weeks]
next_weeks = next_weeks[1:]
next_day = next_weeks[0]
next_day

'2023-04-12'

# Classes and Functions

In [7]:
# get data from the configuration file
def get_data(config, ticker):
    ts = TimeSeries(key=config["alpha_vantage"]["key"]) 
    data, meta_data = ts.get_daily_adjusted(ticker, outputsize=config["alpha_vantage"]["outputsize"])

    date_data = [date for date in data.keys()]
    date_data.reverse()

    close_price_data = [float(data[date][config["alpha_vantage"]["key_adjusted_close"]]) for date in data.keys()]
    close_price_data.reverse()
    close_price_data = np.array(close_price_data)

    num_data_points = len(date_data)
    display_date_range = "from " + date_data[0] + " to " + date_data[num_data_points-1]
    print("Number data points", num_data_points, display_date_range)

    return date_data, close_price_data, num_data_points, display_date_range

class Normalization():
    def __init__(self):
        self.mu = None
        self.sd = None

    def fit_transform(self, x):
        self.mu = np.mean(x, axis=(0), keepdims=True)
        self.sd = np.std(x, axis=(0), keepdims=True)
        normalized_x = (x - self.mu)/self.sd
        return normalized_x

    def inverse_transform(self, x):
        return (x*self.sd) + self.mu
    
def prepare_data_x(x, window_size):
    # perform windowing
    n_row = x.shape[0] - window_size + 1
    output = np.lib.stride_tricks.as_strided(x, shape=(n_row, window_size), strides=(x.strides[0], x.strides[0]))
    return output[:-1], output[-1]


def prepare_data_y(x, window_size):
    # use the next day as label
    output = x[window_size:]
    return output

# Class to prepare data for training and LSTM model
class TimeSeriesDataset(Dataset):
    def __init__(self, x, y):
        x = np.expand_dims(x, 2) # right now we have only 1 feature, so we need to convert `x` into [batch, sequence, features]
        self.x = x.astype(np.float32)
        self.y = y.astype(np.float32)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

# neural network model definition
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=32, num_layers=2, output_size=1, dropout=0.2):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.linear_1 = nn.Linear(input_size, hidden_layer_size)
        self.relu = nn.ReLU()
        self.lstm = nn.LSTM(hidden_layer_size, hidden_size=self.hidden_layer_size, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(num_layers*hidden_layer_size, output_size)
        
        self.init_weights()

    def init_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                 nn.init.constant_(param, 0.0)
            elif 'weight_ih' in name:
                 nn.init.kaiming_normal_(param)
            elif 'weight_hh' in name:
                 nn.init.orthogonal_(param)

    def forward(self, x):
        batchsize = x.shape[0]

        # layer 1
        x = self.linear_1(x)
        x = self.relu(x)
        
        # LSTM layer
        lstm_out, (h_n, c_n) = self.lstm(x)

        # reshape output from hidden cell into [batch, features] for `linear_2`
        x = h_n.permute(1, 0, 2).reshape(batchsize, -1) 
        
        # layer 2
        x = self.dropout(x)
        predictions = self.linear_2(x)
        return predictions[:,-1]
    
# function for training LSTM model
def run_epoch(dataloader, is_training=False):
    epoch_loss = 0

    if is_training:
        model.train()
    else:
        model.eval()

    for idx, (x, y) in enumerate(dataloader):
        if is_training:
            optimizer.zero_grad()

        batchsize = x.shape[0]

        x = x.to(config["training"]["device"])
        y = y.to(config["training"]["device"])

        out = model(x)
        loss = criterion(out.contiguous(), y.contiguous())

        if is_training:
            loss.backward()
            optimizer.step()

        epoch_loss += (loss.detach().item() / batchsize)

    lr = scheduler.get_last_lr()[0]

    return epoch_loss, lr


## Predict the Next Day

In [8]:
# API call limit per minute
limit = 5

check = time.time()

for i,ticker in enumerate(tickers):
    
    # account for API calls per minute
    if not i + 1 % limit:
        sleep = check + 60 - time.time()
        if sleep > 0:
            time.sleep(sleep)
        check = time.time()      
    
    date_data, close_price_data, num_data_points, display_date_range = get_data(config, ticker)
    
    scaler = Normalization()
    normalized_close_price_data = scaler.fit_transform(close_price_data)
    
    data_x, data_x_unseen = prepare_data_x(normalized_close_price_data, window_size=config["data"]["window_size"])
    data_y = prepare_data_y(normalized_close_price_data, window_size=config["data"]["window_size"])
    
    split_index = int(data_y.shape[0]*config["data"]["train_split_size"])
    data_x_train = data_x[:split_index]
    data_x_val = data_x[split_index:]
    data_y_train = data_y[:split_index]
    data_y_val = data_y[split_index:]

    train_dataset = TimeSeriesDataset(data_x_train, data_y_train)
    train_dataloader = DataLoader(train_dataset, batch_size=config["training"]["batch_size"], shuffle=True)

    model = LSTMModel(input_size=config["model"]["input_size"], hidden_layer_size=config["model"]["lstm_size"], num_layers=config["model"]["num_lstm_layers"], output_size=1, dropout=config["model"]["dropout"])
    model = model.to(config["training"]["device"])

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config["training"]["learning_rate"], betas=(0.9, 0.98), eps=1e-9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=config["training"]["scheduler_step_size"], gamma=0.1)
    
    print('{} training'.format(ticker))
    for epoch in range(config["training"]["num_epoch"]):
        loss_train, lr_train = run_epoch(train_dataloader, is_training=True)
        scheduler.step()

        print('Epoch[{}/{}] | loss train:{:.6f}| lr:{:.6f}'
                  .format(epoch+1, config["training"]["num_epoch"], loss_train, lr_train))
        
    model.eval()
    
    torch.tensor(data_x_unseen)
    x = torch.tensor(data_x_unseen).float().to(config["training"]["device"]).unsqueeze(0).unsqueeze(2) # this is the data type and shape required, [batch, sequence, feature]
    prediction = model(x)
    prediction = prediction.cpu().detach().numpy()
    prediction[0] = scaler.inverse_transform(prediction[0])
    
    df = pd.DataFrame([[next_day,prediction[0]]], columns = ['date','close price']) 
    path = "csv/TimeSeries/" + ticker + "_predict.csv"
    df.to_csv(path)


Number data points 5898 from 1999-11-01 to 2023-04-11
MMM training
Epoch[1/50] | loss train:0.069969| lr:0.010000
Epoch[2/50] | loss train:0.014881| lr:0.010000
Epoch[3/50] | loss train:0.012386| lr:0.010000
Epoch[4/50] | loss train:0.011412| lr:0.010000
Epoch[5/50] | loss train:0.012905| lr:0.010000
Epoch[6/50] | loss train:0.010018| lr:0.010000
Epoch[7/50] | loss train:0.009381| lr:0.010000
Epoch[8/50] | loss train:0.011195| lr:0.010000
Epoch[9/50] | loss train:0.009559| lr:0.010000
Epoch[10/50] | loss train:0.009274| lr:0.010000
Epoch[11/50] | loss train:0.009353| lr:0.010000
Epoch[12/50] | loss train:0.009010| lr:0.010000
Epoch[13/50] | loss train:0.009488| lr:0.010000
Epoch[14/50] | loss train:0.010056| lr:0.010000
Epoch[15/50] | loss train:0.009801| lr:0.010000
Epoch[16/50] | loss train:0.008470| lr:0.010000
Epoch[17/50] | loss train:0.009286| lr:0.010000
Epoch[18/50] | loss train:0.008665| lr:0.010000
Epoch[19/50] | loss train:0.008970| lr:0.010000
Epoch[20/50] | loss train:0.00

Epoch[17/50] | loss train:0.004369| lr:0.010000
Epoch[18/50] | loss train:0.005590| lr:0.010000
Epoch[19/50] | loss train:0.004902| lr:0.010000
Epoch[20/50] | loss train:0.004978| lr:0.010000
Epoch[21/50] | loss train:0.005248| lr:0.010000
Epoch[22/50] | loss train:0.004427| lr:0.010000
Epoch[23/50] | loss train:0.004493| lr:0.010000
Epoch[24/50] | loss train:0.004554| lr:0.010000
Epoch[25/50] | loss train:0.004792| lr:0.010000
Epoch[26/50] | loss train:0.004584| lr:0.010000
Epoch[27/50] | loss train:0.004565| lr:0.010000
Epoch[28/50] | loss train:0.004891| lr:0.010000
Epoch[29/50] | loss train:0.004160| lr:0.010000
Epoch[30/50] | loss train:0.004206| lr:0.010000
Epoch[31/50] | loss train:0.004554| lr:0.010000
Epoch[32/50] | loss train:0.005025| lr:0.010000
Epoch[33/50] | loss train:0.006256| lr:0.010000
Epoch[34/50] | loss train:0.004713| lr:0.010000
Epoch[35/50] | loss train:0.004068| lr:0.010000
Epoch[36/50] | loss train:0.004165| lr:0.010000
Epoch[37/50] | loss train:0.004092| lr:0

Epoch[35/50] | loss train:0.009487| lr:0.010000
Epoch[36/50] | loss train:0.009826| lr:0.010000
Epoch[37/50] | loss train:0.011619| lr:0.010000
Epoch[38/50] | loss train:0.011795| lr:0.010000
Epoch[39/50] | loss train:0.010426| lr:0.010000
Epoch[40/50] | loss train:0.010437| lr:0.010000
Epoch[41/50] | loss train:0.008591| lr:0.001000
Epoch[42/50] | loss train:0.007959| lr:0.001000
Epoch[43/50] | loss train:0.007753| lr:0.001000
Epoch[44/50] | loss train:0.007804| lr:0.001000
Epoch[45/50] | loss train:0.007855| lr:0.001000
Epoch[46/50] | loss train:0.007344| lr:0.001000
Epoch[47/50] | loss train:0.007720| lr:0.001000
Epoch[48/50] | loss train:0.007374| lr:0.001000
Epoch[49/50] | loss train:0.007159| lr:0.001000
Epoch[50/50] | loss train:0.007696| lr:0.001000
Number data points 5898 from 1999-11-01 to 2023-04-11
ADBE training
Epoch[1/50] | loss train:0.069146| lr:0.010000
Epoch[2/50] | loss train:0.015342| lr:0.010000
Epoch[3/50] | loss train:0.013493| lr:0.010000
Epoch[4/50] | loss trai

Epoch[1/50] | loss train:0.070596| lr:0.010000
Epoch[2/50] | loss train:0.020252| lr:0.010000
Epoch[3/50] | loss train:0.018669| lr:0.010000
Epoch[4/50] | loss train:0.016366| lr:0.010000
Epoch[5/50] | loss train:0.016298| lr:0.010000
Epoch[6/50] | loss train:0.015387| lr:0.010000
Epoch[7/50] | loss train:0.015738| lr:0.010000
Epoch[8/50] | loss train:0.016795| lr:0.010000
Epoch[9/50] | loss train:0.016089| lr:0.010000
Epoch[10/50] | loss train:0.015515| lr:0.010000
Epoch[11/50] | loss train:0.014696| lr:0.010000
Epoch[12/50] | loss train:0.013947| lr:0.010000
Epoch[13/50] | loss train:0.014115| lr:0.010000
Epoch[14/50] | loss train:0.015557| lr:0.010000
Epoch[15/50] | loss train:0.014628| lr:0.010000
Epoch[16/50] | loss train:0.014674| lr:0.010000
Epoch[17/50] | loss train:0.013351| lr:0.010000
Epoch[18/50] | loss train:0.014555| lr:0.010000
Epoch[19/50] | loss train:0.015536| lr:0.010000
Epoch[20/50] | loss train:0.013454| lr:0.010000
Epoch[21/50] | loss train:0.015425| lr:0.010000
E

Epoch[19/50] | loss train:0.010817| lr:0.010000
Epoch[20/50] | loss train:0.010205| lr:0.010000
Epoch[21/50] | loss train:0.008902| lr:0.010000
Epoch[22/50] | loss train:0.009480| lr:0.010000
Epoch[23/50] | loss train:0.009244| lr:0.010000
Epoch[24/50] | loss train:0.009539| lr:0.010000
Epoch[25/50] | loss train:0.009031| lr:0.010000
Epoch[26/50] | loss train:0.008605| lr:0.010000
Epoch[27/50] | loss train:0.009467| lr:0.010000
Epoch[28/50] | loss train:0.011356| lr:0.010000
Epoch[29/50] | loss train:0.008857| lr:0.010000
Epoch[30/50] | loss train:0.008659| lr:0.010000
Epoch[31/50] | loss train:0.009042| lr:0.010000
Epoch[32/50] | loss train:0.008776| lr:0.010000
Epoch[33/50] | loss train:0.009506| lr:0.010000
Epoch[34/50] | loss train:0.009256| lr:0.010000
Epoch[35/50] | loss train:0.008724| lr:0.010000
Epoch[36/50] | loss train:0.009174| lr:0.010000
Epoch[37/50] | loss train:0.009263| lr:0.010000
Epoch[38/50] | loss train:0.009633| lr:0.010000
Epoch[39/50] | loss train:0.008372| lr:0

...

Epoch[25/50] | loss train:0.009437| lr:0.010000
Epoch[26/50] | loss train:0.011077| lr:0.010000
Epoch[27/50] | loss train:0.010059| lr:0.010000
Epoch[28/50] | loss train:0.010848| lr:0.010000
Epoch[29/50] | loss train:0.010288| lr:0.010000
Epoch[30/50] | loss train:0.010446| lr:0.010000
Epoch[31/50] | loss train:0.010770| lr:0.010000
Epoch[32/50] | loss train:0.010030| lr:0.010000
Epoch[33/50] | loss train:0.010240| lr:0.010000
Epoch[34/50] | loss train:0.009721| lr:0.010000
Epoch[35/50] | loss train:0.010197| lr:0.010000
Epoch[36/50] | loss train:0.010769| lr:0.010000
Epoch[37/50] | loss train:0.010222| lr:0.010000
Epoch[38/50] | loss train:0.010265| lr:0.010000
Epoch[39/50] | loss train:0.010030| lr:0.010000
Epoch[40/50] | loss train:0.010327| lr:0.010000
Epoch[41/50] | loss train:0.008365| lr:0.001000
Epoch[42/50] | loss train:0.008845| lr:0.001000
Epoch[43/50] | loss train:0.008237| lr:0.001000
Epoch[44/50] | loss train:0.008455| lr:0.001000
Epoch[45/50] | loss train:0.008480| lr:0

Epoch[43/50] | loss train:0.008454| lr:0.001000
Epoch[44/50] | loss train:0.008440| lr:0.001000
Epoch[45/50] | loss train:0.008339| lr:0.001000
Epoch[46/50] | loss train:0.008584| lr:0.001000
Epoch[47/50] | loss train:0.007782| lr:0.001000
Epoch[48/50] | loss train:0.008071| lr:0.001000
Epoch[49/50] | loss train:0.008298| lr:0.001000
Epoch[50/50] | loss train:0.008316| lr:0.001000
Number data points 5898 from 1999-11-01 to 2023-04-11
WMB training
Epoch[1/50] | loss train:0.072101| lr:0.010000
Epoch[2/50] | loss train:0.016085| lr:0.010000
Epoch[3/50] | loss train:0.014725| lr:0.010000
Epoch[4/50] | loss train:0.014681| lr:0.010000
Epoch[5/50] | loss train:0.014247| lr:0.010000
Epoch[6/50] | loss train:0.013509| lr:0.010000
Epoch[7/50] | loss train:0.013540| lr:0.010000
Epoch[8/50] | loss train:0.012668| lr:0.010000
Epoch[9/50] | loss train:0.012709| lr:0.010000
Epoch[10/50] | loss train:0.013274| lr:0.010000
Epoch[11/50] | loss train:0.012501| lr:0.010000
Epoch[12/50] | loss train:0.01

Epoch[9/50] | loss train:0.014246| lr:0.010000
Epoch[10/50] | loss train:0.012021| lr:0.010000
Epoch[11/50] | loss train:0.011628| lr:0.010000
Epoch[12/50] | loss train:0.012046| lr:0.010000
Epoch[13/50] | loss train:0.012172| lr:0.010000
Epoch[14/50] | loss train:0.011323| lr:0.010000
Epoch[15/50] | loss train:0.011353| lr:0.010000
Epoch[16/50] | loss train:0.011633| lr:0.010000
Epoch[17/50] | loss train:0.011446| lr:0.010000
Epoch[18/50] | loss train:0.011398| lr:0.010000
Epoch[19/50] | loss train:0.011418| lr:0.010000
Epoch[20/50] | loss train:0.011426| lr:0.010000
Epoch[21/50] | loss train:0.011614| lr:0.010000
Epoch[22/50] | loss train:0.011605| lr:0.010000
Epoch[23/50] | loss train:0.010492| lr:0.010000
Epoch[24/50] | loss train:0.011634| lr:0.010000
Epoch[25/50] | loss train:0.011648| lr:0.010000
Epoch[26/50] | loss train:0.010946| lr:0.010000
Epoch[27/50] | loss train:0.011275| lr:0.010000
Epoch[28/50] | loss train:0.010800| lr:0.010000
Epoch[29/50] | loss train:0.011290| lr:0.

Epoch[27/50] | loss train:0.008594| lr:0.010000
Epoch[28/50] | loss train:0.009012| lr:0.010000
Epoch[29/50] | loss train:0.009911| lr:0.010000
Epoch[30/50] | loss train:0.008346| lr:0.010000
Epoch[31/50] | loss train:0.008077| lr:0.010000
Epoch[32/50] | loss train:0.008434| lr:0.010000
Epoch[33/50] | loss train:0.008774| lr:0.010000
Epoch[34/50] | loss train:0.008356| lr:0.010000
Epoch[35/50] | loss train:0.008372| lr:0.010000
Epoch[36/50] | loss train:0.009359| lr:0.010000
Epoch[37/50] | loss train:0.008295| lr:0.010000
Epoch[38/50] | loss train:0.008978| lr:0.010000
Epoch[39/50] | loss train:0.007938| lr:0.010000
Epoch[40/50] | loss train:0.008188| lr:0.010000
Epoch[41/50] | loss train:0.007492| lr:0.001000
Epoch[42/50] | loss train:0.007010| lr:0.001000
Epoch[43/50] | loss train:0.007423| lr:0.001000
Epoch[44/50] | loss train:0.007060| lr:0.001000
Epoch[45/50] | loss train:0.006885| lr:0.001000
Epoch[46/50] | loss train:0.006777| lr:0.001000
Epoch[47/50] | loss train:0.006768| lr:0

Epoch[45/50] | loss train:0.010436| lr:0.001000
Epoch[46/50] | loss train:0.010817| lr:0.001000
Epoch[47/50] | loss train:0.010349| lr:0.001000
Epoch[48/50] | loss train:0.010411| lr:0.001000
Epoch[49/50] | loss train:0.010727| lr:0.001000
Epoch[50/50] | loss train:0.010076| lr:0.001000
Number data points 2565 from 2013-02-01 to 2023-04-11
ZTS training
Epoch[1/50] | loss train:0.057234| lr:0.010000
Epoch[2/50] | loss train:0.010175| lr:0.010000
Epoch[3/50] | loss train:0.006948| lr:0.010000
Epoch[4/50] | loss train:0.006089| lr:0.010000
Epoch[5/50] | loss train:0.004731| lr:0.010000
Epoch[6/50] | loss train:0.005995| lr:0.010000
Epoch[7/50] | loss train:0.004954| lr:0.010000
Epoch[8/50] | loss train:0.005135| lr:0.010000
Epoch[9/50] | loss train:0.005543| lr:0.010000
Epoch[10/50] | loss train:0.004630| lr:0.010000
Epoch[11/50] | loss train:0.005104| lr:0.010000
Epoch[12/50] | loss train:0.005122| lr:0.010000
Epoch[13/50] | loss train:0.005247| lr:0.010000
Epoch[14/50] | loss train:0.00

In [9]:
end = time.time()
print("Program finished in {} seconds".format(end-start))

Program finished in 46798.09273433685 seconds


In [10]:
predictions = pd.DataFrame(columns = ['date','ticker','close price'])

for ticker in tickers:
    path = "csv/TimeSeries/" + ticker + "_predict.csv"
    df1 = pd.read_csv(path)
    data = {'date': [df1['date'][0]],
            'ticker': [ticker],
            'close price': [df1['close price'][0]]}
    df2 = pd.DataFrame(data)
    predictions = pd.concat([predictions,df2], ignore_index=True)
    
predictions.to_csv("csv/predictions.csv")